# Movie Sentiment Analysis Web App



In [1]:
# Making sure that SageMaker 1.x is installed
!pip install sagemaker==1.72.0

You should consider upgrading via the '/home/ec2-user/anaconda3/envs/pytorch_p36/bin/python -m pip install --upgrade pip' command.


## Step 1: Downloading the data

We will be using the [IMDb dataset](http://ai.stanford.edu/~amaas/data/sentiment/)

> Maas, Andrew L., et al. [Learning Word Vectors for Sentiment Analysis](http://ai.stanford.edu/~amaas/data/sentiment/). In _Proceedings of the 49th Annual Meeting of the Association for Computational Linguistics: Human Language Technologies_. Association for Computational Linguistics, 2011.

In [2]:
%mkdir ../data
!wget -O ../data/aclImdb_v1.tar.gz http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
!tar -zxf ../data/aclImdb_v1.tar.gz -C ../data

mkdir: cannot create directory ‘../data’: File exists
--2020-12-19 13:41:40--  http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
Resolving ai.stanford.edu (ai.stanford.edu)... 171.64.68.10
Connecting to ai.stanford.edu (ai.stanford.edu)|171.64.68.10|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 84125825 (80M) [application/x-gzip]
Saving to: ‘../data/aclImdb_v1.tar.gz’

../data/aclImdb_v1. 100%[===================>]  80.23M  5.95MB/s    in 19s     

2020-12-19 13:42:00 (4.18 MB/s) - ‘../data/aclImdb_v1.tar.gz’ saved [84125825/84125825]



## Step 2: Preparing and Processing the data

To begin with, we will read in each of the reviews and combine them into a single input structure. Then, we will split the dataset into a training set and a testing set.

In [3]:
import os
import glob

def read_imdb_data(data_dir='../data/aclImdb'):
    data = {}
    labels = {}
    
    for data_type in ['train', 'test']:
        data[data_type] = {}
        labels[data_type] = {}
        
        for sentiment in ['pos', 'neg']:
            data[data_type][sentiment] = []
            labels[data_type][sentiment] = []
            
            path = os.path.join(data_dir, data_type, sentiment, '*.txt')
            files = glob.glob(path)
            
            for f in files:
                with open(f) as review:
                    data[data_type][sentiment].append(review.read())
                    # Here we represent a positive review by '1' and a negative review by '0'
                    labels[data_type][sentiment].append(1 if sentiment == 'pos' else 0)
                    
            assert len(data[data_type][sentiment]) == len(labels[data_type][sentiment]), \
                    "{}/{} data size does not match labels size".format(data_type, sentiment)
                
    return data, labels

In [4]:
data, labels = read_imdb_data()
print("IMDB reviews: train = {} pos / {} neg, test = {} pos / {} neg".format(
            len(data['train']['pos']), len(data['train']['neg']),
            len(data['test']['pos']), len(data['test']['neg'])))

IMDB reviews: train = 12500 pos / 12500 neg, test = 12500 pos / 12500 neg


Now that we've read the raw training and testing data from the downloaded dataset, we will combine the positive and negative reviews and shuffle the resulting records.

In [5]:
from sklearn.utils import shuffle

def prepare_imdb_data(data, labels):
    """Prepare training and test sets from IMDb movie reviews."""
    
    #Combine positive and negative reviews and labels
    data_train = data['train']['pos'] + data['train']['neg']
    data_test = data['test']['pos'] + data['test']['neg']
    labels_train = labels['train']['pos'] + labels['train']['neg']
    labels_test = labels['test']['pos'] + labels['test']['neg']
    
    #Shuffle reviews and corresponding labels within training and test sets
    data_train, labels_train = shuffle(data_train, labels_train)
    data_test, labels_test = shuffle(data_test, labels_test)
    
    # Return a unified training data, test data, training labels, test labets
    return data_train, data_test, labels_train, labels_test

In [6]:
train_X, test_X, train_y, test_y = prepare_imdb_data(data, labels)
print("IMDb reviews (combined): train = {}, test = {}".format(len(train_X), len(test_X)))

IMDb reviews (combined): train = 25000, test = 25000


Checking the data

In [7]:
print(train_X[100])
print(train_y[100])

Did anyone stop to realise what sort of movie they were producing here ? Now let`s a former marine officer becomes assinged to a group of kids at a cadet school so this should be a family comedy right ? Wrong . This is just a gross comedy aimed at teenagers with many bad taste moments .It might have been watchable in an extremely dumb way at this point but I found Damon Wayans voice to be irritating beyond belief . Does he speak like that in real life ? If he does then he has my sympathy but he won`t be getting any of my money from watching his movies
0


Removing any HTML tags and tokenizing our input.

In [8]:
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import *

import re
from bs4 import BeautifulSoup

def review_to_words(review):
    nltk.download("stopwords", quiet=True)
    stemmer = PorterStemmer()
    
    text = BeautifulSoup(review, "html.parser").get_text() # Remove HTML tags
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower()) # Convert to lower case
    words = text.split() # Split string into words
    words = [w for w in words if w not in stopwords.words("english")] # Remove stopwords
    words = [PorterStemmer().stem(w) for w in words] # stem
    
    return words

In [9]:
# Checking one of the inputs
review_to_words(train_X[50])

['canadian',
 'filmmak',
 'mari',
 'harron',
 'cultur',
 'gadfli',
 'whose',
 'previou',
 'film',
 'laid',
 'bare',
 'artist',
 'excess',
 'sixti',
 'hollow',
 'avarici',
 'eighti',
 'notori',
 'betti',
 'page',
 'point',
 'unswerv',
 'eye',
 'fifti',
 'america',
 'era',
 'cloak',
 'moral',
 'righteous',
 'joe',
 'mccarthi',
 'experienc',
 'begin',
 'sexual',
 'awaken',
 'would',
 'result',
 'free',
 'love',
 'next',
 'decad',
 'harron',
 'co',
 'writer',
 'guinever',
 'turner',
 'clearli',
 'interest',
 'standard',
 'biopic',
 'sex',
 'symbol',
 'film',
 'underground',
 'icon',
 'era',
 'pure',
 'unasham',
 'sexual',
 'reveal',
 'predatori',
 'instinct',
 'impur',
 'thought',
 'cultur',
 'untouch',
 'beauti',
 'nude',
 'bodi',
 'detail',
 'betti',
 'life',
 'film',
 'concern',
 'end',
 'tragic',
 'period',
 'begin',
 'clearli',
 'harron',
 'interest',
 'america',
 'attitud',
 'toward',
 'sexual',
 'imageri',
 'togeth',
 'fearless',
 'lead',
 'perform',
 'gretchen',
 'mol',
 'stunningl

Applying the review_to_words methods to each of the reviews and caching the results so that we do not have to do this processing each time we run this notebook as it takes a long time to process

In [10]:
import pickle

cache_dir = os.path.join("../cache", "sentiment_analysis")  # where to store cache files
os.makedirs(cache_dir, exist_ok=True)  # ensure cache directory exists

def preprocess_data(data_train, data_test, labels_train, labels_test,
                    cache_dir=cache_dir, cache_file="preprocessed_data.pkl"):
    """Convert each review to words; read from cache if available."""

    # If cache_file is not None, try to read from it first
    cache_data = None
    if cache_file is not None:
        try:
            with open(os.path.join(cache_dir, cache_file), "rb") as f:
                cache_data = pickle.load(f)
            print("Read preprocessed data from cache file:", cache_file)
        except:
            pass  # unable to read from cache
    
    # If cache is missing, then do the heavy lifting
    if cache_data is None:
        
        # Preprocess training and test data to obtain words for each review
        words_train = [review_to_words(review) for review in data_train]
        words_test = [review_to_words(review) for review in data_test]
        
        # Write to cache file for future runs
        if cache_file is not None:
            cache_data = dict(words_train=words_train, words_test=words_test,
                              labels_train=labels_train, labels_test=labels_test)
            with open(os.path.join(cache_dir, cache_file), "wb") as f:
                pickle.dump(cache_data, f)
            print("Wrote preprocessed data to cache file:", cache_file)
    else:
        # Unpack data loaded from cache file
        words_train, words_test, labels_train, labels_test = (cache_data['words_train'],
                cache_data['words_test'], cache_data['labels_train'], cache_data['labels_test'])
    
    return words_train, words_test, labels_train, labels_test

In [11]:
# Preprocess data
train_X, test_X, train_y, test_y = preprocess_data(train_X, test_X, train_y, test_y)

Read preprocessed data from cache file: preprocessed_data.pkl


## Transform the data

To start, we will represent each word as an integer. Of course, some of the words that appear in the reviews occur very infrequently and so likely don't contain much information for the purposes of sentiment analysis. The way we will deal with this problem is that we will fix the size of our working vocabulary and we will only include the words that appear most frequently. We will then combine all of the infrequent words into a single category and, in our case, we will label it as `1`.

Since we will be using a recurrent neural network, it will be convenient if the length of each review is the same. To do this, we will fix a size for our reviews and then pad short reviews with the category 'no word' (which we will label `0`) and truncate long reviews.



### Creating a word dictionary

To begin with, we need to construct a way to map words that appear in the reviews to integers. Here we fix the size of our vocabulary (including the 'no word' and 'infrequent' categories) to be `5000`.

Note that even though the vocab_size is set to `5000`, we only want to construct a mapping for the most frequently appearing `4998` words. This is because we want to reserve the special labels `0` for 'no word' and `1` for 'infrequent word'.

In [12]:
import numpy as np
from collections import Counter 

def build_dict(data, vocab_size = 5000):
    """Construct and return a dictionary mapping each of the most frequently appearing words to a unique integer."""
   
    # A dict storing the words that appear in the reviews along with how often they occur
    word_count = {} 
    
    for review in data:
        for word in review:
            if word not in word_count.keys():
                word_count[word] = 1
            else:
                word_count[word] +=1
    
    sorted_words = sorted(word_count, key=word_count.get,reverse = True)
    
    word_dict = {} # This is what we are building, a dictionary that translates words into integers
    for idx, word in enumerate(sorted_words[:vocab_size - 2]): # The -2 is so that we save room for the 'no word'
        word_dict[word] = idx + 2                              # 'infrequent' labels
        
    return word_dict

In [13]:
word_dict = build_dict(train_X)

### Saving `word_dict`

Saving the word_dict for future use especially when submitting reviews to the deployed endpoint.


In [15]:
data_dir = '../data/pytorch' # The folder we will use for storing data
if not os.path.exists(data_dir): 
    os.makedirs(data_dir)

In [16]:
with open(os.path.join(data_dir, 'word_dict.pkl'), "wb") as f:
    pickle.dump(word_dict, f)

### Transform the reviews

Converting our reviews to their integer sequence representation. Also, padding or truncating to a fixed length, which in our case is `500`.

In [17]:
def convert_and_pad(word_dict, sentence, pad=500):
    NOWORD = 0 # Using 0 to represent the 'no word' category
    INFREQ = 1 # Using 1 to represent the infrequent words, i.e., words not appearing in word_dict
    
    working_sentence = [NOWORD] * pad
    
    for word_index, word in enumerate(sentence[:pad]):
        if word in word_dict:
            working_sentence[word_index] = word_dict[word]
        else:
            working_sentence[word_index] = INFREQ
            
    return working_sentence, min(len(sentence), pad)

def convert_and_pad_data(word_dict, data, pad=500):
    result = []
    lengths = []
    
    for sentence in data:
        converted, leng = convert_and_pad(word_dict, sentence, pad)
        result.append(converted)
        lengths.append(leng)
        
    return np.array(result), np.array(lengths)

In [18]:
train_X, train_X_len = convert_and_pad_data(word_dict, train_X)
test_X, test_X_len = convert_and_pad_data(word_dict, test_X)

In [19]:
# Making sure everything is working as intended.
train_X[1]


array([ 207,   26,  505,   36, 1144,    4, 3871,  846, 1529,  207,  152,
        760,  514,  315,   69,  595,  406, 1091,  531,    1,    2,  146,
          1,  723, 2811,  723,  199, 2811,   62,  199, 2245, 1195, 3439,
        760,  152,   82,   62,    5,   82,  591,    1,  579,  304,   82,
        406,  108,   46,  797,  123,  190,   25,   37,    2,   76,   26,
       1310,    1,  736,  626,  760,    1, 3311,    1, 4356,  220,  319,
          1,    1,    2, 3112,  653,  564,   46,   71,  505,   52,    6,
         50,  224, 2793,  612,    1,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,   

## Uploading the data to S3

Uploading the training dataset to S3 in order for the training code to access it. We will save it locally and then upload to S3 bucket.

### Save the processed training dataset locally

Format of the dataset: each row of the dataset has the form `label`, `length`, `review[500]` where `review[500]` is a sequence of `500` integers representing the words in the review.

In [20]:
import pandas as pd
    
pd.concat([pd.DataFrame(train_y), pd.DataFrame(train_X_len), pd.DataFrame(train_X)], axis=1) \
        .to_csv(os.path.join(data_dir, 'train.csv'), header=False, index=False)

### Uploading the training data


In [21]:
import sagemaker

sagemaker_session = sagemaker.Session()

bucket = sagemaker_session.default_bucket()
prefix = 'sagemaker/sentiment_rnn'

role = sagemaker.get_execution_role()

In [22]:
input_data = sagemaker_session.upload_data(path=data_dir, bucket=bucket, key_prefix=prefix)

The cell above uploads the entire contents of our data directory. This includes the `word_dict.pkl` file. 

## Build and Train the PyTorch Model


In [23]:
!pygmentize train/model.py

import torch.nn as nn

class LSTMClassifier(nn.Module):
    """
    This is the simple RNN model we will be using to perform Sentiment Analysis.
    """

    def __init__(self, embedding_dim, hidden_dim, vocab_size):
        """
        Initialize the model by settingg up the various layers.
        """
        super(LSTMClassifier, self).__init__()

        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim)
        self.dense = nn.Linear(in_features=hidden_dim, out_features=1)
        self.sig = nn.Sigmoid()
        
        self.word_dict = None

    def forward(self, x):
        """
        Perform a forward pass of our model on some input.
        """
        x = x.t()
        lengths = x[0,:]
        reviews = x[1:,:]
        embeds = self.embedding(reviews)
        lstm_out, _ = self.lstm(embeds)
        out = self.dense(lstm_out)
        out = out[lengths - 1, range(len(lengths))]
        return self.sig

In [24]:
import torch
import torch.utils.data

# Reading only the first 250 rows to check if model is working fine or not
train_sample = pd.read_csv(os.path.join(data_dir, 'train.csv'), header=None, names=None, nrows=250)

# Turning input pandas dataframe into tensors
train_sample_y = torch.from_numpy(train_sample[[0]].values).float().squeeze()
train_sample_X = torch.from_numpy(train_sample.drop([0], axis=1).values).long()

# Building the dataset and the dataloader
train_sample_ds = torch.utils.data.TensorDataset(train_sample_X, train_sample_y)
train_sample_dl = torch.utils.data.DataLoader(train_sample_ds, batch_size=50)

### Training method

This will be copied in the train.py file if the implementation is correct.

In [25]:
def train(model, train_loader, epochs, optimizer, loss_fn, device):
    for epoch in range(1, epochs + 1):
        model.train()
        total_loss = 0
        for batch in train_loader:         
            batch_X, batch_y = batch
            
            batch_X = batch_X.to(device)
            batch_y = batch_y.to(device)
            
            model.zero_grad()

            # Calculating Output
            output = model(batch_X)

            # calculateing the loss and performing backpropogation
            loss = loss_fn(output, batch_y)
            loss.backward()
         
            optimizer.step()
            total_loss += loss.data.item()
        print("Epoch: {}, BCELoss: {}".format(epoch, total_loss / len(train_loader)))

Testing the traing method

In [26]:
import torch.optim as optim
from train.model import LSTMClassifier

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = LSTMClassifier(32, 100, 5000).to(device)
optimizer = optim.Adam(model.parameters())
loss_fn = torch.nn.BCELoss()

train(model, train_sample_dl, 5, optimizer, loss_fn, device)

Epoch: 1, BCELoss: 0.6921862006187439
Epoch: 2, BCELoss: 0.6827911019325257
Epoch: 3, BCELoss: 0.6746416926383972
Epoch: 4, BCELoss: 0.6655132174491882
Epoch: 5, BCELoss: 0.6543623328208923




Constructing the PyTorch model using Sagemaker by providing the training script. Required Python Libraries are listed in `requirements.txt`.


### Training the model



In [27]:
from sagemaker.pytorch import PyTorch

estimator = PyTorch(entry_point="train.py",
                    source_dir="train",
                    role=role,
                    framework_version='0.4.0',
                    train_instance_count=1,
                    train_instance_type='ml.p2.xlarge',
                    hyperparameters={
                        'epochs': 10,
                        'hidden_dim': 200,
                    })

In [28]:
estimator.fit({'training': input_data})

'create_image_uri' will be deprecated in favor of 'ImageURIProvider' class in SageMaker Python SDK v2.
's3_input' class will be renamed to 'TrainingInput' in SageMaker Python SDK v2.
'create_image_uri' will be deprecated in favor of 'ImageURIProvider' class in SageMaker Python SDK v2.


2020-12-19 13:43:19 Starting - Starting the training job...
2020-12-19 13:43:21 Starting - Launching requested ML instances......
2020-12-19 13:44:25 Starting - Preparing the instances for training.........
2020-12-19 13:46:05 Downloading - Downloading input data......
2020-12-19 13:47:11 Training - Training image download completed. Training in progress.bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2020-12-19 13:47:12,406 sagemaker-containers INFO     Imported framework sagemaker_pytorch_container.training
2020-12-19 13:47:12,431 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2020-12-19 13:47:15,450 sagemaker_pytorch_container.training INFO     Invoking user training script.
2020-12-19 13:47:15,715 sagemaker-containers INFO     Module train does not provide a setup.py. 
Generating setup.py
2020-12-19 13:47:15,716 sagemaker-containers INFO     Generating setup.cfg
2020-12-19 13:47


##  Deploying the model for testing


In [2]:
!pygmentize train/train.py

import argparse
import json
import os
import pickle
import sys
import sagemaker_containers
import pandas as pd
import torch
import torch.optim as optim
import torch.utils.data

from model import LSTMClassifier

def model_fn(model_dir):
    
    print("Loading model.")

    # First, load the parameters used to create the model.
    model_info = {}
    model_info_path = os.path.join(model_dir, 'model_info.pth')
    with open(model_info_path, 'rb') as f:
        model_info = torch.load(f)

    print("model_info: {}".format(model_info))

    # Determine the device and construct the model.
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = LSTMClassifier(model_info['embedding_dim'], model_info['hidden_dim'], model_info['vocab_size'])

    # Load the stored model parameters.
    model_path = os.path.join(model_dir, 'model.pth')
    with open(model_path, 'rb') as f:
        model.load_state_dict(torch.load(f))

    # Load the saved word_dict.
    word_dict_p

In [30]:
predictor = estimator.deploy(initial_instance_count=1, instance_type='ml.m4.xlarge')

Parameter image will be renamed to image_uri in SageMaker Python SDK v2.
'create_image_uri' will be deprecated in favor of 'ImageURIProvider' class in SageMaker Python SDK v2.


-------------!

##  Using the model for testing



In [31]:
test_X = pd.concat([pd.DataFrame(test_X_len), pd.DataFrame(test_X)], axis=1)
print(test_X.values)

[[  20  385   84 ...    0    0    0]
 [ 145   17  171 ...    0    0    0]
 [  35   29    2 ...    0    0    0]
 ...
 [ 210   76  210 ...    0    0    0]
 [  19   36  139 ...    0    0    0]
 [  34 4889  930 ...    0    0    0]]


In [32]:

def predict(data, rows=512):
    split_array = np.array_split(data, int(data.shape[0] / float(rows) + 1))
    predictions = np.array([])
    for array in split_array:
        
        predictions = np.append(predictions, predictor.predict(array))
    
    return predictions

In [33]:
predictions = predict(test_X.values)
predictions = [round(num) for num in predictions]

In [34]:
from sklearn.metrics import accuracy_score
accuracy_score(test_y, predictions)

0.8542

### Testing a User Provided Review

In [35]:
test_review = 'The simplest pleasures in life are the best, and this film is one of them. Combining a rather basic storyline of love and adventure this movie transcends the usual weekend fair with wit and unmitigated charm.'


In [36]:
# Preprocessing and Predicting
test_data, test_data_len = convert_and_pad(word_dict,review_to_words(test_review))
test_data = pd.concat([pd.DataFrame([test_data_len]), pd.DataFrame([test_data])], axis=1)


In [37]:
prediction = predict(test_data.values)
print(prediction)

[0.92920458]


It is returning a value close to one, hence the prediction is correct -- Positive.

### Delete the endpoint
Another Endpoint will be deployed for using in the webapp.

In [38]:
estimator.delete_endpoint()

estimator.delete_endpoint() will be deprecated in SageMaker Python SDK v2. Please use the delete_endpoint() function on your predictor instead.


##  Deploying the model for the web app

Deployed PyTorch model uses the following functions from the predict.py file.

 - `model_fn`: This function is the same function that we used in the training script and it tells SageMaker how to load our model.
 - `input_fn`: This function receives the raw serialized input that has been sent to the model's endpoint and its job is to de-serialize and make the input available for the inference code.
 - `output_fn`: This function takes the output of the inference code and its job is to serialize this output and return it to the caller of the model's endpoint.
 - `predict_fn`: The heart of the inference script, this is where the actual prediction is done and is the function which you will need to complete.



In [1]:
!pygmentize serve/predict.py

import argparse
import json
import os
import pickle
import sys
import sagemaker_containers
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data

from model import LSTMClassifier

from utils import review_to_words, convert_and_pad

def model_fn(model_dir):
    
    print("Loading model.")

    # Loading the parameters used to create the model.
    model_info = {}
    model_info_path = os.path.join(model_dir, 'model_info.pth')
    with open(model_info_path, 'rb') as f:
        model_info = torch.load(f)

    print("model_info: {}".format(model_info))

    # Determining the device and constructing the model.
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = LSTMClassifier(model_info['embedding_dim'], model_info['hidden_dim'], model_info['vocab_size'])

    
    model_path = os.path.join(model_dir, 'model.pth')
    with open(model_path, 'rb') as f:
        model.load_state_dict(torc

### Deploying the model


In [40]:
from sagemaker.predictor import RealTimePredictor
from sagemaker.pytorch import PyTorchModel

class StringPredictor(RealTimePredictor):
    def __init__(self, endpoint_name, sagemaker_session):
        super(StringPredictor, self).__init__(endpoint_name, sagemaker_session, content_type='text/plain')

model = PyTorchModel(model_data=estimator.model_data,
                     role = role,
                     framework_version='0.4.0',
                     entry_point='predict.py',
                     source_dir='serve',
                     predictor_cls=StringPredictor)
predictor = model.deploy(initial_instance_count=1, instance_type='ml.m4.xlarge')

Parameter image will be renamed to image_uri in SageMaker Python SDK v2.
'create_image_uri' will be deprecated in favor of 'ImageURIProvider' class in SageMaker Python SDK v2.


---------------!

### Testing the model


In [41]:
import glob

def test_reviews(data_dir='../data/aclImdb', stop=250):
    
    results = []
    ground = []
    
    # We make sure to test both positive and negative reviews    
    for sentiment in ['pos', 'neg']:
        
        path = os.path.join(data_dir, 'test', sentiment, '*.txt')
        files = glob.glob(path)
        
        files_read = 0
        
        print('Starting ', sentiment, ' files')
        
        # Iterate through the files and send them to the predictor
        for f in files:
            with open(f) as review:
                # First, we store the ground truth (was the review positive or negative)
                if sentiment == 'pos':
                    ground.append(1)
                else:
                    ground.append(0)
                # Read in the review and convert to 'utf-8' for transmission via HTTP
                review_input = review.read().encode('utf-8')

                # Send the review to the predictor and store the results
                results.append(float(predictor.predict(review_input)))
                
            # Sending reviews to our endpoint one at a time takes a while so we
            # only send a small number of reviews
            files_read += 1
            if files_read == stop:
                break
            
    return ground, results

In [43]:
ground, results = test_reviews()

Starting  pos  files
Starting  neg  files


In [44]:
from sklearn.metrics import accuracy_score
accuracy_score(ground, results)

0.876

As an additional test, we can try sending the `test_review` that we looked at earlier.

In [42]:
predictor.predict(test_review)

b'1'

## Using the model for the web app

After Creating the Endpoint, following steps were carried out:

1. Creating an IAM role for the Lambda Function.
2. Creating the Lambda Function.
3. Setting up an API gateway with POST method to be called in the index.html file.




In [45]:
predictor.endpoint

'sagemaker-pytorch-2020-12-19-14-03-09-881'

In [ ]:
import boto3

#Lambda function to be added in AWS
def lambda_handler(event, context):

    # The SageMaker runtime is what allows us to invoke the endpoint that we've created.
    runtime = boto3.Session().client('sagemaker-runtime')

    # Now we use the SageMaker runtime to invoke our endpoint, sending the review we were given
    response = runtime.invoke_endpoint(EndpointName = 'sagemaker-pytorch-2020-12-19-14-03-09-881',    
                                       ContentType = 'text/plain',                 
                                       Body = event['body'])                       

    # The response is an HTTP response whose body contains the result of our inference
    result = response['Body'].read().decode('utf-8')

    return {
        'statusCode' : 200,
        'headers' : { 'Content-Type' : 'text/plain', 'Access-Control-Allow-Origin' : '*' },
        'body' : result
    }

### Delete the endpoint

Deleting Endpoint when not using.

In [46]:
predictor.delete_endpoint()